In [1]:
!pip install transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 53.6 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 514.8/514.8 kB 41.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 45.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 59.7 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 69.7 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface_hub 0.29.2
    Uninstalling huggingface_hub-0.29.2:
      Successfully uninstalled huggingface_hub-0.29.2
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2


In [2]:
from transformers import AutoTokenizer
tokenizer=AutoTokenizer.from_pretrained("gpt2")
ids=tokenizer("night in the city of London",return_tensors="pt").input_ids
ids

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

tensor([[3847,  287,  262, 1748,  286, 3576]])

In [3]:
for t in ids[0]:
    print(t,":\t",tokenizer.decode(t))

tensor(3847) :	 night
tensor(287) :	  in
tensor(262) :	  the
tensor(1748) :	  city
tensor(286) :	  of
tensor(3576) :	  London


In [4]:
from transformers import AutoModelForCausalLM
gpt2=AutoModelForCausalLM.from_pretrained("gpt2",pad_token_id=tokenizer.eos_token_id)
outputs=gpt2(ids)
outputs.logits.shape

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

torch.Size([1, 6, 50257])

In [5]:
final_logits=gpt2(ids).logits[0][-1]
final_logits.argmax()

tensor(13)

In [6]:
tokenizer.decode(final_logits.argmax())

'.'

In [8]:
import torch
top10_logits=torch.topk(final_logits,10)
for index in top10_logits.indices:
    print(tokenizer.decode(index))

.
,
 and
 in
 on


 is
 where
 was
."


In [10]:
if tokenizer.pad_token is None:
    tokenizer.pad_token=tokenizer.eos_token #EndOfStatement token
output_ids=gpt2.generate(ids,max_new_tokens=10,pad_token_id=tokenizer.pad_token_id)
decoded_text=tokenizer.decode(output_ids[0])
print("Input IDs:",ids[0])
print("Output IDs:",output_ids[0])
print(f"Generated text: {decoded_text}")

Input IDs: tensor([3847,  287,  262, 1748,  286, 3576])
Output IDs: tensor([3847,  287,  262, 1748,  286, 3576,   13,  198,  198,  464,  582,   11,
         508,  318,  407, 3706])
Generated text: night in the city of London.

The man, who is not named
